## Installing packages

TODO:


*   Check for the number problem again
*   Write a code to get random 1000 samples ( currently it is taking first 100)
*   Clean up code. Remove redundancieas, write imports properly
*   To increase run time, instead of repeatedly downloading the data set we can locally save it and just load it.
*   Similarly also maay load the output locally
*   Evaluation metric
      *    Bleu metric and meteor

when i tried this thing in english it is adding random number. Maybe it does the same in french?
* The reopening of the legislative period - english of the french
* Resumption of the session - english of the german


In [3]:
!pip install datasets transformers

## Making imports

In [4]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

## Importing and pre-processing the dataset

In [5]:
# Load the Europarl dataset
dataset = load_dataset("europarl_bilingual", "de-fr")

# Inspect the dataset
print(dataset)
required_subset_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
df = pd.DataFrame(required_subset_dataset)


#Making a 80-20 Split of the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/76.3k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1942666 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1942666
    })
})


In [16]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: Tesla T4


## now we look into importing the model

In [17]:
model_name = "Qwen/Qwen2-1.5B"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Define the function for translation

In [52]:
def translate_to_french(input_sentence):
    # Construct the prompt for translation
    prompt = f"Translate the following text from German into French: \"{input_sentence}\".\nOutput (French):"

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the output
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,  # Limit the length of the generated output
            do_sample=False,     # Disable sampling to ensure deterministic output
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the output tokens to text
    translated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the translated part from the model's output
    # Assuming the model appends the translation after "Translation: "
    start_index = translated_text.find('Output (French): "') + len('Output (French): "')
    end_index = translated_text.find('"', start_index)
    translated_text = translated_text[start_index:].strip()

    translated_text = translated_text.split("\n")[0].strip()
    translated_text = translated_text.strip('".')
    return translated_text

In [50]:
#Example usage
input_sentence = required_subset_dataset[0]["translation"]["de"]
translation = translate_to_french(input_sentence)
print(f"{translation}")

L'introduction du droit d'occupation permanente est un exemple concret


In [27]:
required_subset_dataset[0]["translation"]["fr"]

"Le droit de séjour permanent, qui est à l'étude, en constitue un exemple concret."

In [19]:
def translate_test_set(test_data):
    translations = []
    for i, row in test_data.iterrows():
          german_sentence = row['translation']['de']
          french = translate_to_french(german_sentence)
          translations.append({
              'original_de': german_sentence,
              'translation': french,
          })
          print(f"Translated {i + 1}/{len(test_data)}: {german_sentence} -> {french}")
    return translations


TODO: This part looking wrong

In [20]:
subset_size = 10  # we need 1000 (randomly smaple 1000)
translated_results = translate_test_set(test_df[:subset_size])

# Save the results to a file
translated_df = pd.DataFrame(translated_results)
translated_df.to_csv("translated_test_set.csv", index=False, encoding="utf-8")

print("Translation completed and saved to 'translated_test_set.csv'.")

Translated 522/10: Ich bin für die Zusammenarbeit zwischen unabhängigen Staaten. -> "Je suis pour la coopération entre les États indépendants."
Translated 738/10: Außerdem stimme ich allen zu, die erklärt haben, es sei sehr wichtig, guten Beziehungen zu diesem äußerst wichtigen Land, das wirtschaftlich, technisch und politisch immer mehr an Bedeutung gewinnt, Vorrang zu geben. -> "En outre, je suis d'accord avec tous ceux qui ont déclaré que c'était très important de donner une priorité aux relations avec ce pays extrêmement important, qui est économique, technologique et politique toujours plus en valeur.".
Translated 741/10: Ich hoffe, dass diese Änderungsvorschläge die Unterstützung des Hauses finden. -> "Je souhaite que ces propositions d'amélioration trouvent l'appui du Parlement."
Translated 661/10: Fragestunde (Anfragen an den Rat) -> "Questions de tableaux (Questions à l'Assemblée)".
Translated 412/10: Wir sagen gleichzeitig nein zu Vereinfachungen und zu Diskussionen symbolisc